In [ ]:
# range: 1987-07-02 to 2011-12-31
# leap years: 1988, 1992, 1996, 2000, 2004, 2008
# number of days per month
# (1,31), (2,28), (3,31), (4,30), (5,31), (6,30)
# (7,31), (8,31), (9,30), (10,31), (11,30), (12,31)
# earth circumference: 40,075 km
# 0<lat<pi, 0<lon<2pi

import numpy as np
from scipy.interpolate import RectSphereBivariateSpline
from scipy.io import netcdf
import matplotlib.pyplot as plt
from matplotlib import animation
%matplotlib inline
import cartopy.crs as ccrs
import cartopy.feature as cartopy_features
from datetime import date, datetime, timedelta
from os import remove

PI = np.pi

data_root = '/home/joe/data/'
file_pre = data_root + 'podaac-ftp.jpl.nasa.gov/allData/ccmp/L3.0/flk/'

netcdf_on_local_disk = False

day = timedelta(days=1)
min_date, max_date = date(1987,7,2), date(2011,12,31)
n_days = (max_date-min_date).days
n_lat, n_lon= 628, 1440
min_lat_centered, max_lat_centered = -78.375, 78.375
min_lon_centered, max_lon_centered = .125-180, 359.875-180
min_lat, max_lat = 90-78.375, 90+78.375
min_lon, max_lon = .125, 359.875
min_lat_rad = PI*min_lat/180
max_lat_rad = PI*max_lat/180
min_lon_rad = PI*min_lon/180
max_lon_rad = PI*max_lon/180
earth_radius = 6371.008 # km

In [3]:
# range: 1987-07-02 to 2011-12-31
# leap years: 1988, 1992, 1996, 2000, 2004, 2008
# number of days per month
# (1,31), (2,28), (3,31), (4,30), (5,31), (6,30)
# (7,31), (8,31), (9,30), (10,31), (11,30), (12,31)

# plot vector fields on globe, movie!
# interpolate space for fixed time point
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
%matplotlib inline
from scipy.interpolate import RectSphereBivariateSpline
from scipy.io import netcdf
from datetime import date, timedelta
from os import remove

data_root = '/home/joe/data/'
file_pre = data_root + 'podaac-ftp.jpl.nasa.gov/allData/ccmp/L3.0/flk/'
netcdf_on_local_disk = False

day = timedelta(days=1)
min_date, max_date = date(1987,7,2), date(2011,12,31)
n_days = (max_date-min_date).days
n_lat, n_lon = 628, 1440
min_lat, max_lat = -78.375, 78.375
min_lon, max_lon = .125, 359.875
min_lat_rescale = np.pi*(min_lat+90)/180
min_lon_rescale = np.pi*min_lon/180
max_lat_rescale = np.pi*(max_lat+90)/180
max_lon_rescale = np.pi*max_lon/180

In [4]:
if netcdf_on_local_disk:
    # return netcdf file for given date
    file_post = '_v11l30flk.nc'
    def date_to_cdf(d):
        filename = d.strftime('%Y/%m') + '/analysis_' + d.strftime('%Y%m%d')
        filename = file_pre + filename + file_post
        cdf_file = netcdf.netcdf_file(filename)
        return cdf_file
    def cdf_to_npz(d):
        cdf_file = date_to_cdf(d)
        uwnd = np.copy(cdf_file.variables['uwnd'].data)
        vwnd = np.copy(cdf_file.variables['vwnd'].data)
        time = np.copy(cdf_file.variables['time'].data)
        filename = file_pre + d.strftime('%Y/%m/%Y%m%d.npz')
        np.savez(filename,uwnd=uwnd,vwnd=vwnd,time=time)
        cdf_file.close()
else:
    # return lon/lat files
    def lat_lon(rescale=False):
        # lat, lon = file_pre + 'lat.npy', file_pre + 'lon.npy'
        # lat, lon = np.load(lat), np.load(lon)
        lat = np.linspace(-78.375,78.375,n_lat)
        lon = np.linspace(.125,359.875,n_lon)
        if rescale:
            # 0<lat<pi, 0<lon<2pi
            lat = np.pi*(lat+90)/180
            lon = np.pi*lon/180
        return lat, lon
    # return npz files for  given date
    def date_to_npz(d):
        filename = file_pre + d.strftime('%Y/%m/%Y%m%d.npz')
        with np.load(filename) as data:
            time,uwnd,vwnd = data['time'],data['uwnd'],data['vwnd']
        return time, uwnd, vwnd

In [8]:
lat_spl, lon_spl = lat_lon(rescale=True)
time, uwnd, vwnd = date_to_npz(min_date)
print uwnd[0,0,:10]
print vwnd[0,0,:10]

[-927 -923 -920 -917 -914 -910 -907 -904 -900 -897]
[-1973 -1994 -2014 -2035 -2056 -2077 -2098 -2118 -2139 -2158]


In [15]:
from datetime import date, datetime, timedelta

current_time = datetime(2005,4,13,23,23,43)
print '%02i' % (current_time.hour/6*6,)

18


In [ ]:
import os

current_date = min_date
while current_date < max_date:
    time,u,v = date_to_npz(current_date)
    directory = file_pre + current_date.strftime('%Y/%m/%d/')
    if not os.path.exists(directory):
        os.makedirs(directory)
    for idx in range(4):
        filename = directory + '%02i.npz' % (6*idx,)
        np.savez(filename,u=u[idx],v=v[idx])
    filename = file_pre + current_date.strftime('%Y/%m/%Y%m%d.npz')
    os.remove(filename)
    current_date += day

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = plt.subplot(111, projection=ccrs.PlateCarree())
ax.coastlines()
ax.add_feature(cartopy_features.OCEAN, zorder=0)
ax.add_feature(cartopy_features.LAND, zorder=0, edgecolor='black')
ax.set_global()

min_datetime = datetime(1987,7,2)
max_datetime = datetime(2011,12,31,23,59,59)


# start datetime, end date time, dim of ic grid
current_datetime = start_datetime
# load data for current date
# interpolate u and v for this and next hour
while current_datetime < end_datetime:
    #while current_datetime < appropriate 6 hour mark:
        #integrate forward in time



def integrate(lat,lon):
    sin_lat, cos_lat = np.sin(lat), np.cos(lat)
    sin_lon, cos_lon = np.sin(lon), np.cos(lon)
    p = earth_radius*np.array([cos_lat*cos_lon, cos_lat*sin_lon, sin_lat])
    lat_tan = np.array([-sin_lon, cos_lon, 0])
    lon_tan = np.array([-sin_lat*cos_lon, -sin_lat*sin_lon, cos_lat])
    lat_spl,lon_spl = .5*PI+lat,PI+lon
    if lat_spl < 0 or lat_spl > PI:
        print len(traj[i][j]), lat_spl, lat
    if lon_spl < 0 or lon_spl > 2*PI:
        print len(traj[i][j]), lon_spl, lon    
    dp = 1./60.*(u_spl.ev(lat_spl,lon_spl)*lat_tan + v_spl.ev(lat_spl,lon_spl)*lon_tan)
    p += dp
    p /= np.linalg.norm(p)
    lat, lon = np.arcsin(p[2]), np.arctan2(p[1],p[0])
    lat = max(lat, -78.375/180*PI)
    lat = min(lat, 78.375/180*PI)
    if lon > PI:
        lon-=2*PI
    elif lon < -PI:
        lon+=2*PI
    return lat, lon

# plot vector field
#n_lat_ev,n_lon_ev = 50,50
#lat = np.linspace(min_lat_rad, max_lat_rad, n_lat_ev)
#lon = np.linspace(min_lon_rad, max_lon_rad, n_lon_ev)
#lat, lon = np.meshgrid(lat, lon)
#u = u_spl.ev(lat,lon)
#v = v_spl.ev(lat,lon)
#lat = np.linspace(min_lat_centered,max_lat_centered,n_lat_ev)
#lon = np.linspace(min_lon_centered,max_lon_centered,n_lon_ev)
#lat, lon = np.meshgrid(lat, lon)
#Q = ax.quiver(lon,lat,u,v,transform=ccrs.PlateCarree())

# interpolate
current_date = min_date
time,uwnd,vwnd = date_to_npz(current_date, units='km_per_hr')
lat_spl, lon_spl = lat_lon(units='rad')

u_spl = RectSphereBivariateSpline(lat_spl,lon_spl,uwnd[0])
v_spl = RectSphereBivariateSpline(lat_spl,lon_spl,vwnd[0])

n_lat_int, n_lon_int = 10,10
lat_grid = np.linspace(-78.375/180*PI,78.375/180*PI,n_lat_int)
lon_grid = np.linspace((.125-180)/180*PI,(359.875-180)/180*PI,n_lon_int)
lat2d, lon2d = np.meshgrid(lat_grid,lon_grid)
traj = [['' for j in range(n_lon_int)] for i in range(n_lat_int)]
for i in range(n_lat_int):
    for j in range(n_lon_int):
        p = [lat2d[i,j],lon2d[i,j]]
        traj[i][j] = [p]
        for _ in range(60):
            p = integrate(*p)
            traj[i][j].append(np.copy(p))
        traj[i][j] = np.array(traj[i][j])
        ax.plot(np.rad2deg(traj[i][j][:,1]), np.rad2deg(traj[i][j][:,0]), 'ro', transform=ccrs.PlateCarree())


current_date += day
time,uwnd,vwnd = date_to_npz(current_date, units='km_per_hr')
u_spl = RectSphereBivariateSpline(lat_spl,lon_spl,uwnd[0])
v_spl = RectSphereBivariateSpline(lat_spl,lon_spl,vwnd[0])
for i in range(n_lat_int):
    for j in range(n_lon_int):
        p = traj[i][j][-1]
        traj[i][j] = [p]
        for _ in range(60):
            p = integrate(*p)
            traj[i][j].append(np.copy(p))
        traj[i][j] = np.array(traj[i][j])
        ax.plot(np.rad2deg(traj[i][j][:,1]), np.rad2deg(traj[i][j][:,0]), 'bo', transform=ccrs.PlateCarree())
        
plt.show()

In [ ]:
# spatial interpolate
current_date = min_date+np.random.randint(n_days)*day
time,uwnd,vwnd = date_to_npz(current_date)

lat,lon = lat_lon(units='rad')

u_spl = RectSphereBivariateSpline(lat,lon,uwnd[0])
v_spl = RectSphereBivariateSpline(lat,lon,vwnd[0])

lat, lon = np.meshgrid(lat, lon)
u = u_spl.ev(lat,lon).T
v = v_spl.ev(lat,lon).T

fig,ax = plt.subplots(2,2)
ax[0,0].imshow(uwnd[0], interpolation='nearest',origin='bottom')
ax[0,1].imshow(u, interpolation='nearest',origin='bottom')
ax[1,0].imshow(vwnd[0], interpolation='nearest',origin='bottom')
ax[1,1].imshow(v, interpolation='nearest',origin='bottom')

In [ ]:
# return lat/lon files
def lat_lon(units='deg'):
    # lat, lon = file_pre + 'lat.npy', file_pre + 'lon.npy'
    # lat, lon = np.load(lat), np.load(lon)
    lat = np.linspace(min_lat,max_lat,n_lat)
    lon = np.linspace(min_lon,max_lon,n_lon)
    if units == 'rad':
        lat = PI*lat/180
        lon = PI*lon/180
    return lat, lon
# return npz files for given date
def date_to_npz(d, units='m_per_s'):
    filename = file_pre + d.strftime('%Y/%m/%Y%m%d.npz')
    with np.load(filename) as data:
        time,uwnd,vwnd = data['time'],data['uwnd'],data['vwnd']
        if units == 'km_per_hr':
            uwnd = 3.6*np.array(uwnd)
            vwnd = 3.6*np.array(vwnd)
    return time, uwnd, vwnd